In [41]:
import pandas as pd
import numpy as np
from pybaseball import pitching_stats
from pybaseball import playerid_reverse_lookup

import requests
from bs4 import BeautifulSoup


In [148]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

## Pitching Data

#### Load Data

In [42]:
# Read in pitching data from pybaseball
data = pitching_stats(2010,2024,qual=50,ind=1)

In [43]:
# Replace FLA abbreviation with MIA for the Marlins
data['Team'].replace('FLA', 'MIA', inplace=True)

In [44]:
data[data['Name'] == 'Yoshihisa Hirano']

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
3771,19756,2019,Yoshihisa Hirano,ARI,35,5,5,0.5,4.75,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,19756,2018,Yoshihisa Hirano,ARI,34,4,3,0.2,2.44,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Subset data to only necessary columns 
sub = data[['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'GS', 'CG', 'IP', 'TBF', 'Pitches', 
             'FA% (sc)', 'FT% (sc)', 'FC% (sc)', 'FS% (sc)', 'FO% (sc)', 'SI% (sc)', 'SL% (sc)', 'CU% (sc)', 'KC% (sc)', 'EP% (sc)', 'CH% (sc)', 'SC% (sc)', 'KN% (sc)', 'UN% (sc)',
             'vFA (sc)', 'vFT (sc)', 'vFC (sc)', 'vFS (sc)', 'vFO (sc)', 'vSI (sc)', 'vSL (sc)', 'vCU (sc)', 'vKC (sc)', 'vEP (sc)', 'vCH (sc)', 'vSC (sc)', 'vKN (sc)',
             'FA-X (sc)', 'FT-X (sc)', 'FC-X (sc)', 'FS-X (sc)', 'FO-X (sc)', 'SI-X (sc)', 'SL-X (sc)', 'CU-X (sc)', 'KC-X (sc)', 'EP-X (sc)', 'CH-X (sc)', 'SC-X (sc)', 'KN-X (sc)',
             'FA-Z (sc)', 'FT-Z (sc)', 'FC-Z (sc)', 'FS-Z (sc)', 'FO-Z (sc)', 'SI-Z (sc)', 'SL-Z (sc)', 'CU-Z (sc)', 'KC-Z (sc)', 'EP-Z (sc)', 'CH-Z (sc)', 'SC-Z (sc)', 'KN-Z (sc)',
             'Pace']]

In [46]:
# Get list of Fangraphs IDs
fg_ids = data['IDfg'].tolist()

In [47]:
# Get ID lookup table
id_table = playerid_reverse_lookup(fg_ids, key_type='fangraphs')

In [48]:
# Merge Retrosheet key to dataset
sub.rename(columns={'IDfg': 'key_fangraphs'}, inplace=True)
sub = sub.merge(id_table[['key_retro', 'key_fangraphs']], on=['key_fangraphs'], how='left')

/Users/swarg22/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


#### Create Lag Variables

In [49]:
sub = sub.sort_values(['Name', 'Age'])

In [50]:
# Lag total number of pitches for each pitcher over the previouse 1, 2, and 3 seasons
sub['pitches_1y'] = sub.groupby('Name')['Pitches'].apply(lambda x: x.rolling(1, min_periods=1, closed='left').sum())
sub['pitches_2y'] = sub.groupby('Name')['Pitches'].apply(lambda x: x.rolling(2, min_periods=1, closed='left').sum())
sub['pitches_3y'] = sub.groupby('Name')['Pitches'].apply(lambda x: x.rolling(3, min_periods=1, closed='left').sum())

In [51]:
# Lag total number of innings for each pitcher over the previouse 1, 2, and 3 seasons
sub['innings_1y'] = sub.groupby('Name')['IP'].apply(lambda x: x.rolling(1, min_periods=1, closed='left').sum())
sub['innings_2y'] = sub.groupby('Name')['IP'].apply(lambda x: x.rolling(2, min_periods=1, closed='left').sum())
sub['innings_3y'] = sub.groupby('Name')['IP'].apply(lambda x: x.rolling(3, min_periods=1, closed='left').sum())

#### Clean Data

In [52]:
# Read in Retrosheet bio data
url = 'https://raw.githubusercontent.com/chadwickbureau/retrosheet/refs/heads/master/reference/biofile.csv'
bio = pd.read_csv(url)

In [53]:
# Merge MLB debut date into dataset
bio.rename(columns={'PLAYERID': 'key_retro'}, inplace=True)
debuted = sub.merge(bio[['key_retro', 'PLAY.DEBUT']], on=['key_retro'], how='left')

In [54]:
# Filter data to only players that debuted during the 2010 season or later
debuted['PLAY.DEBUT'] = pd.to_datetime(debuted['PLAY.DEBUT'])
debuted = debuted[debuted['PLAY.DEBUT'] > '2010-01-01']

In [55]:
# Fill NAs with 0
debuted.fillna(0, inplace=True)

## Pitcher Injuries

#### Load Data

In [56]:
# Read in pitcher injury data from Pro Sports Transactions

# Base URL
url_base = "https://www.prosportstransactions.com/baseball/Search/SearchResults.php?Player=&Team=&BeginDate=2010-04-04&EndDate=2024-09-27&DLChkBx=yes&submit=Search&start={}"

# Initialize an empty list to store dataframes
dfs = []

npages=901

# Iterate through the page numbers
for i in range(0, npages*25+1, 25): 
    url = url_base.format(i)
    
    # Request the page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table using XPath equivalent in BeautifulSoup
    table = soup.select_one('div:nth-of-type(4) > table:nth-of-type(1)')
    
    if table:
        df = pd.read_html(str(table))[0]  # Read the table into a DataFrame
        df.columns = df.iloc[0]
        df.drop(df.index[0], inplace=True)
        dfs.append(df)

# Concatenate all DataFrames
testing = pd.concat(dfs, ignore_index=True)

# Clean the DataFrame
testing['Acquired'] = testing['Acquired'].str[2:]  # Remove the first 2 characters
testing['Relinquished'] = testing['Relinquished'].str[2:]  # Remove the first 2 characters

#### Clean Data

In [82]:
# Filter data to only elbow injuries
elbow_injuries = testing[testing['Notes'].str.contains('elbow', case=False)]
elbow_injuries = elbow_injuries[~elbow_injuries['Relinquished'].isna()]
elbow_injuries.drop('Acquired',axis=1,inplace=True)

In [83]:
# Clean up parentheses in Relinquished column 
elbow_injuries['Relinquished'] = elbow_injuries['Relinquished'].str.replace(r'\s*\(.*?\)', '', regex=True)

In [84]:
# Split the names by ' / ', returning NaN if there's no split
split_names = elbow_injuries['Relinquished'].str.split(' / ', expand=True)

# Create two DataFrames: one for the left side (or original names) and one for the right side
left_names = elbow_injuries.copy()
left_names['Name'] = split_names[0]

right_names = elbow_injuries.copy()
right_names['Name'] = split_names[1]

# Concatenate the two DataFrames, and drop rows with NaN in the 'Names' column
elbow_injuries = pd.concat([left_names, right_names.dropna()], ignore_index=True)

In [85]:
elbow_injuries.drop('Relinquished',axis=1,inplace=True)

In [86]:
elbow_injuries['Date'] = pd.to_datetime(elbow_injuries['Date'])
elbow_injuries['Season'] = elbow_injuries['Date'].dt.year

I now have 2 Datasets:

Elbow Injuries

In [118]:
elbow_injuries.head()

,Date,Team,Notes,Name,Season
0,2010-04-04,Dodgers,placed on 15-day DL with left elbow injury / l...,Hung-Chih Kuo,2010
1,2010-04-04,Nationals,placed on 15-day DL with strained right elbow,Chris Coste,2010
2,2010-04-04,Pirates,placed on 15-day DL with strained right elbow,Joel Hanrahan,2010
3,2010-04-04,Rays,placed on 15-day DL with strained right elbow,Matt Joyce,2010
4,2010-04-08,Athletics,transferred to 60-day DL recovering from ligam...,Josh Outman,2010


Main Clustering Dataset

In [119]:
debuted.head()

,key_fangraphs,Season,Name,Team,Age,G,GS,CG,IP,TBF,...,Pace,key_retro,pitches_1y,pitches_2y,pitches_3y,innings_1y,innings_2y,innings_3y,PLAY.DEBUT,Best_Match
6,11467,2017,A.J. Cole,WSN,25,11,8,0,52.0,229,...,23.4,colea002,0.0,0.0,0.0,0.0,0.0,0.0,2015-04-28,None
7,11132,2012,A.J. Griffin,OAK,24,15,15,0,82.1,336,...,17.8,grifa002,0.0,0.0,0.0,0.0,0.0,0.0,2012-06-24,A.J. Griffin
8,11132,2013,A.J. Griffin,OAK,25,32,32,1,200.0,823,...,18.0,grifa002,1421.0,1421.0,1421.0,82.1,82.1,82.1,2012-06-24,A.J. Griffin
9,11132,2016,A.J. Griffin,TEX,28,23,23,0,119.0,509,...,19.9,grifa002,3212.0,4633.0,4633.0,200.0,282.1,282.1,2012-06-24,A.J. Griffin
10,11132,2017,A.J. Griffin,TEX,29,18,15,1,77.1,338,...,19.4,grifa002,2063.0,5275.0,6696.0,119.0,319.0,401.1,2012-06-24,A.J. Griffin


What do I want to do?

I want to cluster the main pitcher data. I can do that right now.

However, I need to, after clustering, see which pitcher seasons are near an elbow injury

How can I do this?

I can:
1. Cluster the data now and try to overlay the injuries by changing colors of dots
2. Create a binary indicator column in main data that signifies an injury, cluster, and analyze distribution of this column amongst clusters

In [149]:
##### WORK IN PROGRESS #####

#pip install pandas fuzzywuzzy python-Levenshtein
from fuzzywuzzy import process

# Fuzzy match and create a mapping
matched_names = []
for name in debuted['Name']:
    match = process.extractOne(name, elbow_injuries['Name'])
    matched_names.append(match[0] if match[1] >= 80 else None)
debuted['Matched_Name'] = matched_names

# Merge dataframes on matched names
merged = pd.merge(debuted, elbow_injuries, left_on='Matched_Name', right_on='Name', suffixes=('', '_injury'), how='left')

# Create injury columns using vectorized operations
debuted['injury_curr'] = ((merged['Season_injury'] == merged['Season']).astype(int))
debuted['injury_1y'] = ((merged['Season_injury'] == merged['Season'] - 1).astype(int))
debuted['injury_2y'] = ((merged['Season_injury'] == merged['Season'] - 2).astype(int))

# Clean up the DataFrame
debuted.drop(columns=['Matched_Name'], inplace=True)